In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/My Drive/Colab Notebooks/data/'

In [4]:
df = pd.read_parquet(data_path + "clean_text.parquet")

In [5]:
df

,clean_text,target
13,best pate everhave searching taste since,human
14,pcc great examplereach elected official,human
15,theorem conclusion,human
16,theyre learning best,human
17,thread,human
...,...,...
2353366,ruined everything goodalways said misunderstoo...,human
2353367,every question,human
2353369,song wish someone wrote pt,human
2353371,song wish someone wrote pt,human


In [6]:
X = df.clean_text

y = df['target'].map(lambda x: 1 if x == 'bot' else 0)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
###transforming sentences in word sequences

X_train = [text_to_word_sequence(i) for i in X_train]
X_test = [text_to_word_sequence(i) for i in X_test]

In [11]:
##Tokenizing

tk = Tokenizer()

tk.fit_on_texts(X_train)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

In [12]:
vocab_size = len(tk.word_index)
vocab_size

471139

In [13]:
X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post', maxlen=25)
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post', maxlen=25)

In [14]:
embedding_size = 10

def initialize_model():
    
    model = Sequential()
    
    model.add(layers.Embedding(input_dim=vocab_size+1,output_dim=embedding_size, mask_zero=True, input_length=25))
    
    model.add(layers.Conv1D(20, kernel_size=2))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(10, activation='relu'))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    
  model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy','Precision','Recall']
                 )
    return model

In [15]:
cnn = initialize_model()

cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 10)            4711400   
                                                                 
 conv1d (Conv1D)             (None, 24, 20)            420       
                                                                 
 flatten (Flatten)           (None, 480)               0         
                                                                 
 dense (Dense)               (None, 10)                4810      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 4,716,641
Trainable params: 4,716,641
Non-trainable params: 0
_________________________________________________________________


In [16]:
es = EarlyStopping(patience=5, restore_best_weights=True)

history = cnn.fit(X_train_pad, y_train, 
          epochs=500, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/500
28228/28228 [==============================] - 426s 15ms/step - loss: 0.3269 - accuracy: 0.8875 - precision: 0.8170 - recall: 0.2139 - val_loss: 0.3072 - val_accuracy: 0.8958 - val_precision: 0.8285 - val_recall: 0.2832
Epoch 2/500
28228/28228 [==============================] - 377s 13ms/step - loss: 0.2739 - accuracy: 0.9066 - precision: 0.8471 - recall: 0.3759 - val_loss: 0.3067 - val_accuracy: 0.8976 - val_precision: 0.7870 - val_recall: 0.3267
Epoch 3/500
28228/28228 [==============================] - 379s 13ms/step - loss: 0.2362 - accuracy: 0.9200 - precision: 0.8719 - recall: 0.4768 - val_loss: 0.3371 - val_accuracy: 0.8832 - val_precision: 0.5995 - val_recall: 0.3945
Epoch 4/500
28228/28228 [==============================] - 374s 13ms/step - loss: 0.2094 - accuracy: 0.9299 - precision: 0.8938 - recall: 0.5448 - val_loss: 0.3489 - val_accuracy: 0.8859 - val_precision: 0.6263 - val_recall: 0.3735
Epoch 5/500
28228/28228 [==============================] - 375s 13ms/ste

In [18]:
cnn.evaluate(X_test_pad,y_test)

10082/10082 [==============================] - 52s 5ms/step - loss: 0.3069 - accuracy: 0.8982 - precision: 0.8211 - recall: 0.3130


[0.306924045085907, 0.8982380628585815, 0.8210945725440979, 0.3130362629890442]